In [5]:
import pandas as pd
import json
sample_file = open("/Users/subhakuru/Documents/data/retail_db/schemas.json","r")

schema = json.load(sample_file)
schema


{'departments': [{'column_name': 'department_id',
   'data_type': 'integer',
   'column_position': 1},
  {'column_name': 'department_name',
   'data_type': 'string',
   'column_position': 2}],
 'categories': [{'column_name': 'category_id',
   'data_type': 'integer',
   'column_position': 1},
  {'column_name': 'category_department_id',
   'data_type': 'integer',
   'column_position': 2},
  {'column_name': 'category_name',
   'data_type': 'string',
   'column_position': 3}],
 'orders': [{'column_name': 'order_id',
   'data_type': 'integer',
   'column_position': 1},
  {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
  {'column_name': 'order_customer_id',
   'data_type': 'timestamp',
   'column_position': 3},
  {'column_name': 'order_status',
   'data_type': 'string',
   'column_position': 4}],
 'products': [{'column_name': 'product_id',
   'data_type': 'integer',
   'column_position': 1},
  {'column_name': 'product_cateogry_id',
   'data_type': 'integer',
   'c

In [15]:
def getColumnNames(schema, element, column_name):
    element_details = schema.get(element)
    return [col[column_name] for col in element_details]

In [19]:
order_schema = getColumnNames(schema,"orders","column_name")
customer_schema = getColumnNames(schema, "customers", "column_name")

In [22]:
orders = pd.read_csv("/Users/subhakuru/Documents/data/retail_db/orders/part-00000",names=order_schema)
customers = pd.read_csv("/Users/subhakuru/Documents/data/retail_db/customers/part-00000", names=customer_schema)
orders.head(5)


,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


In [23]:
customers.head(5)

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,725
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,725


In [28]:
customers = customers.set_index('customer_id')
orders = orders.set_index('order_customer_id')

In [29]:
orders.join?


Signature:
orders.join(
    other: 'DataFrame | Series | Iterable[DataFrame | Series]',
    on: 'IndexLabel | None' = None,
    how: 'MergeHow' = 'left',
    lsuffix: 'str' = '',
    rsuffix: 'str' = '',
    sort: 'bool' = False,
    validate: 'str | None' = None,
) -> 'DataFrame'
Docstring:
Join columns of another DataFrame.

Join columns with `other` DataFrame either on index or on a key
column. Efficiently join multiple DataFrame objects by index at once by
passing a list.

Parameters
----------
other : DataFrame, Series, or a list containing any combination of them
    Index should be similar to one of the columns in this one. If a
    Series is passed, its name attribute must be set, and that will be
    used as the column name in the resulting joined DataFrame.
on : str, list of str, or array-like, optional
    Column or index level name(s) in the caller to join on the index
    in `other`, otherwise joins index-on-index. If multiple
    values given, the `other` DataFrame must h

In [34]:
customer_orders = orders.join(customers, how="inner")
customer_orders.head(5)

,order_id,order_date,order_status,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
order_customer_id,,,,,,,,,,,
1,22945,2013-12-13 00:00:00.0,COMPLETE,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
2,15192,2013-10-29 00:00:00.0,PENDING_PAYMENT,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,33865,2014-02-18 00:00:00.0,COMPLETE,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,57963,2013-08-02 00:00:00.0,ON_HOLD,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,67863,2013-11-30 00:00:00.0,COMPLETE,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126


In [36]:
customer_orders.shape

(68883, 11)

In [45]:
customer_orders.groupby('order_customer_id')['order_id'].agg(order_count = 'count' )
#apply()

,order_count
order_customer_id,
1,1
2,4
3,7
4,6
5,4
...,...
12431,16
12432,10
12433,4


In [49]:
orders.sort_values(['order_customer_id','order_date'])

,order_id,order_date,order_status
order_customer_id,,,
1,22945,2013-12-13 00:00:00.0,COMPLETE
2,57963,2013-08-02 00:00:00.0,ON_HOLD
2,15192,2013-10-29 00:00:00.0,PENDING_PAYMENT
2,67863,2013-11-30 00:00:00.0,COMPLETE
2,33865,2014-02-18 00:00:00.0,COMPLETE
...,...,...,...
12434,61777,2013-12-26 00:00:00.0,COMPLETE
12434,42915,2014-04-16 00:00:00.0,COMPLETE
12434,51800,2014-06-14 00:00:00.0,ON_HOLD


In [50]:
orders.sort_values(['order_customer_id','order_date'],ascending=False)

,order_id,order_date,order_status
order_customer_id,,,
12435,41643,2014-04-08 00:00:00.0,PENDING
12435,61629,2013-12-21 00:00:00.0,CANCELED
12434,51800,2014-06-14 00:00:00.0,ON_HOLD
12434,42915,2014-04-16 00:00:00.0,COMPLETE
12434,61777,2013-12-26 00:00:00.0,COMPLETE
...,...,...,...
2,33865,2014-02-18 00:00:00.0,COMPLETE
2,67863,2013-11-30 00:00:00.0,COMPLETE
2,15192,2013-10-29 00:00:00.0,PENDING_PAYMENT


In [51]:
orders.sort_values(['order_customer_id','order_date'],ascending=[True,False])

,order_id,order_date,order_status
order_customer_id,,,
1,22945,2013-12-13 00:00:00.0,COMPLETE
2,33865,2014-02-18 00:00:00.0,COMPLETE
2,67863,2013-11-30 00:00:00.0,COMPLETE
2,15192,2013-10-29 00:00:00.0,PENDING_PAYMENT
2,57963,2013-08-02 00:00:00.0,ON_HOLD
...,...,...,...
12434,5303,2013-08-26 00:00:00.0,PENDING
12434,4799,2013-08-23 00:00:00.0,PENDING_PAYMENT
12434,1868,2013-08-03 00:00:00.0,CLOSED


In [62]:
orders.to_json('result.json',orient="records",lines=True)

In [67]:
pd.read_json("result.json",lines=True)

,order_id,order_date,order_status
0,1,2013-07-25 00:00:00.0,CLOSED
1,2,2013-07-25 00:00:00.0,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,COMPLETE
3,4,2013-07-25 00:00:00.0,CLOSED
4,5,2013-07-25 00:00:00.0,COMPLETE
...,...,...,...
68878,68879,2014-07-09 00:00:00.0,COMPLETE
68879,68880,2014-07-13 00:00:00.0,COMPLETE
68880,68881,2014-07-19 00:00:00.0,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,ON_HOLD
